In [4]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import VotingRegressor

# Load the dataset
file_path = 'bengaluru_house_prices.csv'  # Update this with the correct path if needed
df = pd.read_csv(file_path)

# Display dataset information
print("Dataset loaded successfully!")
print("Shape:", df.shape)
print("Columns:", df.columns)
df.head()


Dataset loaded successfully!
Shape: (13320, 9)
Columns: Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [14]:
pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Handle missing values (if any)
df.fillna(0, inplace=True)

# Feature Engineering
df['price_per_sqft'] = df['price'] / df['area']  # Price per square foot
df['log_area'] = np.log1p(df['area'])           # Log-transform area
df['log_price'] = np.log1p(df['price'])         # Log-transform price

# Features and target
X = df[['area', 'bedroom', 'bathroom', 'price_per_sqft', 'log_area', 'layout_type', 'property_type', 'furnish_type', 'city']]
y = df['log_price']  # Using log-transformed price as the target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data split into training and testing sets successfully!")


KeyError: 'area'

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingRegressor

# Define numerical and categorical features
num_features = ['area', 'bedroom', 'bathroom', 'price_per_sqft', 'log_area']  # Example numerical columns
cat_features = ['layout_type', 'property_type', 'furnish_type', 'city']       # Example categorical columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),  # Scale numerical features
        ('cat', OneHotEncoder(drop='first'), cat_features)  # Encode categorical features
    ]
)

# Verify preprocessing works on data
X_train_processed = preprocessor.fit_transform(X_train)
print("Preprocessed training data shape:", X_train_processed.shape)


NameError: name 'X_train' is not defined

In [11]:
from sklearn.model_selection import GridSearchCV

# Hyperparameter grid
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid search
grid_search_rf = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid_rf,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2
)

# Pipeline
rf_model = Pipeline(steps=[('preprocessor', preprocessor), ('model', grid_search_rf)])
rf_model.fit(X_train, y_train)

# Best parameters
best_rf_params = grid_search_rf.best_params_
print("Best Random Forest Parameters:", best_rf_params)


NameError: name 'X_train' is not defined

In [13]:
# Hyperparameter grid
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Grid search
grid_search_xgb = GridSearchCV(
    estimator=xgb.XGBRegressor(random_state=42),
    param_grid=param_grid_xgb,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2
)

# Pipeline
xgb_model = Pipeline(steps=[('preprocessor', preprocessor), ('model', grid_search_xgb)])
xgb_model.fit(X_train, y_train)

# Best parameters
best_xgb_params = grid_search_xgb.best_params_
print("Best XGBoost Parameters:", best_xgb_params)


NameError: name 'X_train' is not defined

In [15]:
# Define the ensemble model with preprocessing
ensemble_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Add preprocessing
    ('ensemble', VotingRegressor(
        estimators=[
            ('random_forest', grid_search_rf.best_estimator_),
            ('xgboost', grid_search_xgb.best_estimator_)
        ]
    ))
])

# Train the ensemble model
ensemble_pipeline.fit(X_train, y_train)

print("Ensemble model trained successfully!")

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
# Train the ensemble pipeline
ensemble_pipeline.fit(X_train, y_train)
print("Ensemble pipeline trained successfully!")

# Define the evaluation function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    print("Model Evaluation Metrics:")
    print(f"Root Mean Squared Error (RMSE): ₹{rmse:.2f}")
    print(f"R² Score: {r2:.4f}")

# Evaluate the trained pipeline
evaluate_model(ensemble_pipeline, X_test, y_test)


In [19]:
def get_user_input_and_predict_easy():
    """
    Take simplified numeric input from the user and predict the house price.
    """
    print("Enter the following details to predict the house price:\n")

    # Numeric options for layout type
    layout_options = {1: 'BHK', 2: 'RK'}
    print("Layout Type Options:")
    for key, value in layout_options.items():
        print(f"{key}: {value}")
    layout_choice = int(input("\nSelect Layout Type (1/2): "))
    layout_type = layout_options.get(layout_choice, 'BHK')

    # Numeric options for property type
    property_options = {1: 'Apartment', 2: 'Independent House'}
    print("\nProperty Type Options:")
    for key, value in property_options.items():
        print(f"{key}: {value}")
    property_choice = int(input("\nSelect Property Type (1/2): "))
    property_type = property_options.get(property_choice, 'Apartment')

    # Numeric options for furnish type
    furnish_options = {1: 'Furnished', 2: 'Semi-Furnished', 3: 'Unfurnished'}
    print("\nFurnish Type Options:")
    for key, value in furnish_options.items():
        print(f"{key}: {value}")
    furnish_choice = int(input("\nSelect Furnish Type (1/2/3): "))
    furnish_type = furnish_options.get(furnish_choice, 'Furnished')

    # Numeric options for city
    cities = {1: 'Ahmedabad', 2: 'Mumbai', 3: 'Delhi', 4: 'Bangalore', 5: 'Chennai'}
    print("\nCity Options:")
    for key, value in cities.items():
        print(f"{key}: {value}")
    city_choice = int(input("\nSelect City (1/2/3/4/5): "))
    city = cities.get(city_choice, 'Ahmedabad')

    # Other numeric inputs
    bedroom = float(input("\nEnter number of bedrooms (e.g., 2, 3): "))
    area = float(input("Enter area in square feet (e.g., 1200): "))
    bathroom = float(input("Enter number of bathrooms (e.g., 1, 2): "))

    try:
        # Create a dictionary of inputs for prediction
        price_per_sqft = area  # Assuming placeholder
        input_data = {
            'bedroom': [bedroom],
            'layout_type': [layout_type],
            'property_type': [property_type],
            'area': [area],
            'furnish_type': [furnish_type],
            'bathroom': [bathroom],
            'city': [city],
            'price_per_sqft': [price_per_sqft],
            'log_area': [np.log(area)]
        }

        # Convert input data to DataFrame
        input_df = pd.DataFrame(input_data)

        # Ensure input matches preprocessing requirements
        preprocessor = ensemble_pipeline.named_steps['preprocessor']
        ensemble_model = ensemble_pipeline.named_steps['ensemble']

        input_preprocessed = preprocessor.transform(input_df)

        # Predict price using the trained pipeline
        predicted_log_price = ensemble_model.predict(input_preprocessed)[0]
        predicted_price = np.exp(predicted_log_price)  # Reverse log transformation

        print(f"\nPredicted Price: ₹{predicted_price:.2f}")
    except Exception as e:
        print("\nError in prediction. Please check your inputs.")
        print(f"Error details: {e}")


In [ ]:
get_user_input_and_predict_easy()



Enter the following details to predict the house price:

Layout Type Options:
1: BHK
2: RK
